FEATURE IMPORTANCE ANALYSIS

In [ ]:
import pandas as pd
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import os
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [ ]:
# finding present working directory
import os
os.getcwd()

# changing the working directory
os.chdir('/home/researchlab/Downloads/CODES/Data')

print(os.getcwd())

#checking the files in the directory
os.listdir()


In [ ]:
import numpy as np

data1 = pd.read_csv('10_dataset_subset_common.csv')
print(data1)
data1.describe()

# Developing X and Y
X = data1.drop(['ID', 'Label', 'Sequence'], axis=1)
Y = data1[['Label']]
Y = np.ravel(Y)

cols_MI = X.columns



Mutual Information Ranking Analysis

In [ ]:
# Calculating mutual information classification using union dataset
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X, Y, random_state=99)

# sorting the mutual information values in descending order
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns

#plotting the mutual information values
mutual_info = mutual_info.sort_values(ascending=False)
mutual_info.sort_values(ascending=False).plot.bar(figsize=(20,8))

# saving the order of column name to be used in next dataframe
cols_MI_index = mutual_info.index

print(cols_MI_index)

# adding ID, Label, Sequence into the cols_MI_index
cols_MI_index = cols_MI_index.insert(0, 'ID')
cols_MI_index = cols_MI_index.insert(1, 'Label')
cols_MI_index = cols_MI_index.insert(2, 'Sequence')

print(cols_MI_index)


Plots: Matching the dataset with Target information

In [ ]:
# isolating the X columns with maximum value greater than 1.0
df1 = X.loc[:, (X.max() > 1.0)]

#develop a for loop to divide each value in column by 100
for i in df1.columns:
    X[i] = df1[i].div(100)

X.describe()

# combining the X with ID, Label, Sequence column of data1
data1 = pd.concat([data1[['ID', 'Label', 'Sequence']], X], axis=1)
print(data1)


In [ ]:
# setting the index of data1 to cols_MI_index
data1 = data1.reindex(columns=cols_MI_index)
print(data1)

In [ ]:
Seq_target_src_data = pd.read_csv('ahcp_seq_target.csv')
print(Seq_target_src_data.shape)

display(Seq_target_src_data)

t1 = data2.Sequence
t2 = Seq_target_src_data.Sequence
out= t1.map(dict(zip(t2, t2.index)))
print("out:")
display(out)

In [ ]:
tmp_matched_df_Seq_target_src_data = Seq_target_src_data.iloc[out.values]
print("tmp_matched_df_Seq_target_src_data:")
display(tmp_matched_df_Seq_target_src_data[['Sequence', 'Target']])

data2['CheckSeqTarget']=tmp_matched_df_Seq_target_src_data.Sequence.values
data2['Target'] = tmp_matched_df_Seq_target_src_data.Target.values
print("data2:")
display(data2)

if all(data2.CheckSeqTarget.isin(data2.Sequence)):
  print("Check Passed: All Sequences Match!!")
else:
  print("Check Failed: Sequences don't Match")

In [ ]:
data3 = data2.drop(['Label', 'ID', 'Target', 'CheckSeqTarget'], axis=1)
print("data3:")
display(data3)

In [ ]:
out.values

Plots: Generating the Plots

In [ ]:
dirname = "dendograms"

if(os.path.isdir("./" + dirname) == False):
  os.mkdir("./" + dirname)
fignameid1 = "FeatureDistrib_used_Dendograms" + ".png"
fignameid1 = "./" + dirname + "/" + fignameid1
print("Saved file :", fignameid1)


In [ ]:
def show_boxplot(df, title="Selected Features"):
  plt.figure(figsize=(16,3))
  sns.boxplot(data = df, orient="v", palette="Set2", showmeans=True, meanprops={"marker":"o", "markerfacecolor":"white","markeredgecolor":"black", "markersize":"6"})
  plt.title(title, fontsize = 16)
  plt.ylabel("Range", fontweight = 'bold')
  plt.xlabel("Attributes", fontweight = 'bold')
  plt.xticks(rotation=90)
  plt.yscale("log")
  plt.ylim([1e-4, 200])

In [ ]:
show_boxplot(data3)
plt.savefig(fignameid1, dpi=1200, bbox_inches='tight')

In [ ]:
from matplotlib.colors import LogNorm
from matplotlib.patches import Patch
import colorcet as cc

def colourcode_targets(TargetSeries):
  targets_series = TargetSeries
  targetcolours_pal = sns.color_palette(cc.glasbey_dark,n_colors = len(set(targets_series.unique())))
  print(targetcolours_pal)
  targets_lut = dict(zip(targets_series.unique(), targetcolours_pal))
  target_colors = pd.Series(targets_series).map(targets_lut)
  display(targets_series)
  display(targetcolours_pal)
  display(target_colors)

  return target_colors, targets_lut


def change_loc_rowcolors(scm):
  ax_row_colors = scm.ax_row_colors
  box = ax_row_colors.get_position()
  box_heatmap = scm.ax_heatmap.get_position()
  ax_row_colors.set_position([box_heatmap.max[0], box.y0, box.width*1.5, box.height])
  scm.ax_headmap.tick_params(axis='y', pad=25, length=10)


fignameid2 = "OrigData_Clustermap_Features" + ".png"
fignameid2 = "./" + dirname + "/" + fignameid2
print("Saved file:", fignameid2)
sns.set(font_scale=0.6)
ax = plt.figure(figsize=(50,100))

target_colors, targets_lut = colourcode_targets(data2.Target)

In [ ]:
#scm = sns.clustermap(data3.set_index('Sequence'), norm=LogNorm(), cmap='rocket', method='complete', metric='euclidean', xticklabels=True, vmin=1.0e-2, vmax=1.0e2, row_colors=target_colors.values, cbar_pos = (0.3, 0.87, 0.4, 0.02), cbar_kws={"orientation":"horizontal"})
scm = sns.clustermap(data3.set_index('Sequence'), norm=LogNorm(), cmap='rocket', method='complete', metric='euclidean', xticklabels=True, vmin=1.0e-2, vmax=1.0e2, row_colors=target_colors.values, cbar_pos = (0.3, 0.87, 0.4, 0.02), cbar_kws={"orientation":"horizontal"})
scm.ax_col_dendrogram.set_visible(False)
scm.ax_col_dendrogram.set_xlabel('Targets & Features', fontsize=10, fontweight='bold', color='black')

print(scm.ax_heatmap.get_position())
scm.ax_row_colors.set_xticks([scm.ax_heatmap.get_position().max[0] - 0.2])
scm.ax_row_colors.set_xticklabels(["Targets"], rotation=90, fontsize=10, fontweight='bold', color='black')

scm.ax_heatmap.yaxis.set_ticks_position("right")
scm.ax_heatmap.tick_params(axis='x', pad =-5)
for line in scm.ax_heatmap.get_xticklines():
  line.set_data([20,0])

handles = [Patch(facecolor=targets_lut[name]) for name in targets_lut]
plt.legend(handles, targets_lut, title='Targets', bbox_to_anchor=(0.95, 0.2), bbox_transform=plt.gcf().transFigure, loc=(0,40), facecolor='white')
plt.savefig(fignameid2, dpi=1200, bbox_inches='tight')

print(scm.ax_heatmap.yaxis.get_majorticklabels()[2])